# Notebook to Change IC or force function inside the stiff regime of the Duffing Equation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# imports
import time
import torch
import matplotlib.pyplot as plt
import numpy as np
import sys
from scipy.integrate import solve_ivp
from collections import defaultdict
from tqdm import tqdm

# Add parent directory to sys.path
from pathlib import Path
current_path = Path.cwd()
parent_dir = current_path.parent.parent
sys.path.append(str(parent_dir))

# Import necessary modules
from src.utils_plot import plot_loss_and_all_solution
from src.load_save import load_run_history
from src.transfer_learning import compute_H_and_dH_dt, compute_M_inv, compute_force_term
from src.nonlinear_transfer_learning import solve_perturbation_TL

torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

In [ ]:
def check_versions_and_device():
  # set the device to the GPU if it is available, otherwise use the CPU
  current_dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  !nvidia-smi
  return current_dev

# set a global device variable to use in code
dev = check_versions_and_device()
print(dev)

### Load the pretrain model and history

In [ ]:
file_name = "linear_inference_IC_2291457"
equation_name = "duffing"

trained_model, \
x_range, \
iterations, \
hid_lay, \
num_equations, \
num_heads, \
loss_hist, \
alpha_list, \
A_list, \
IC_list, \
force_list = load_run_history(equation_name, file_name, dev, prev=False)

# create the A matrix associated with the ODE
def get_A(alpha):
    return torch.tensor([[0., -1.], [0.1, alpha]], device=dev).double() 

force_list = [
    lambda t: torch.cat([torch.zeros(len(t), device=dev).unsqueeze(1), torch.cos(t).unsqueeze(1)], dim=1).double()
    if not isinstance(t, (float, int))
    else np.array([0, np.cos(t)]).T
    for _ in range(num_heads)
]

# function to numerically compute the solution to any set of two coupled, linear first-order ODES
def double_coupled_equation(t, y, A, force):
    return np.array([force(t)[0] - A[0][1] * y[1] - A[0][0] * y[0],
                     force(t)[1] - A[1][0] * y[0] - A[1][1] * y[1]])

numerical_sol_fct = lambda x, v, A, force: (solve_ivp(double_coupled_equation, [x_range[0], x_range[1]],
                                                    v.squeeze(), args=(A, force), t_eval=x.squeeze(), method="Radau").y)

plot_loss_and_all_solution(x_range=x_range, true_functs=numerical_sol_fct,
                           trained_model=trained_model, IC_list=IC_list, A_list=A_list,
                           force=force_list, train_losses=loss_hist, device=dev)

## Transfer Learning inside the sitff domain
### Extract H 

In [ ]:
# forward pass to extract H, H0 and dH/dt
size = 512
H, H_0, dH_dt_new, t_eval = compute_H_and_dH_dt(x_range[0], x_range[1], trained_model, num_equations, hid_lay, size, dev)
H = H.double()
H_0 = H_0.double()
dH_dt_new = dH_dt_new.double()
t_eval = t_eval.double()

### Equation to solve and stiff domain

In [ ]:
alpha_transfer = 90; beta = 0.5

domain = (x_range[0], x_range[1]);
A_transfer = get_A(alpha=alpha_transfer)

def numerical_non_linear_solution(delta, beta, u0, domain, t_eval, method="Radau", r_tol= 1e-8):
  def F(t, y):
    return [y[1],
            -delta*y[1] - 0.1*y[0] - beta*y[0]**3 + np.cos(t)]
  solution = solve_ivp(F, domain, u0, t_eval=t_eval, method=method, r_tol=r_tol)
  return solution

r_tol = 1e-4
numerical_sol_fct_radau = lambda x, IC, alpha, beta=beta, r_tol=r_tol, domain=domain: numerical_non_linear_solution(alpha, beta, IC, domain, x,
                                                                                                                    method="Radau", r_tol=r_tol).y

numerical_sol_fct_rk45 = lambda x, IC, alpha, beta=beta, r_tol=r_tol, domain=domain: numerical_non_linear_solution(alpha, beta, IC, domain, x,
                                                                                                                   method="RK45", r_tol=r_tol).y

numerical_methods = {"RK45": numerical_sol_fct_rk45, "Radau": numerical_sol_fct_radau}

### Component for perturbation transfer learning

In [ ]:
from src.nonlinear_transfer_learning import solve_perturbation_TL

# functions to calculate the force function of each system p
def force_func_index(n):
    solution_index = [] # ind1, ind2, ind3, coeff
    for a in range(n+1):
        for b in range(a+1):
              for c in range(b+1):
                if ((a+b+c)==n):
                    if ((a==b) & (b==c)):
                        solution_index.append([a, b, c, 1])
                    elif ((a!=b) & (b!=c)):
                        solution_index.append([a, b, c, 6])
                    else:
                        solution_index.append([a, b, c, 3])
    return solution_index

def force_function_PINNS(i, alpha, list_force_index, PINNS_list):
      result = 0
      for force_ind in list_force_index[i-1]:
        result += force_ind[-1]*(PINNS_list[force_ind[0]][:, 0, :]*PINNS_list[force_ind[1]][:, 0, :]*PINNS_list[force_ind[2]][:, 0, :])
      return torch.hstack((torch.zeros_like(result), -result))

def force_function_numerical(i, alpha, list_force_index, numerical_pert_list):
      result = 0
      for force_ind in list_force_index[i-1]:
        result += force_ind[-1]*(numerical_pert_list[force_ind[0]][0, :]*numerical_pert_list[force_ind[1]][0, :]*numerical_pert_list[force_ind[2]][0, :])
      return np.vstack((np.zeros_like(result), -result)).T


def solve_numericaly_perturbation(t, y, A, force, t_eval):
    index = np.argmin(np.abs(t_eval- t))
    return np.array([force[index, 0] - A[0][1] * y[1] - A[0][0] * y[0],
                     force[index, 1] - A[1][0] * y[0] - A[1][1] * y[1]])

numerical_perturbation_fct = lambda x, v, A, force: (solve_ivp(solve_numericaly_perturbation, [x_range[0], x_range[1]],
                                                    v.squeeze(), args=(A, force, x), t_eval=x.squeeze(), method="Radau"))


### Change IC in a stiff regime

In [ ]:
change_force = False
change_IC = True

# training IC (random or not)
def random_IC(x_bound=[1, 3], y_bound=[0, 2]):
    ICx = np.random.uniform(x_bound[0], x_bound[1], 1)
    ICy = np.random.uniform(y_bound[0], y_bound[1], 1)
    return torch.tensor([ICx, ICy], device=dev)

# training force (random or not)
def random_force(force1_bound=[0, 2], force2_bound=[0, 2]):
    force1 = np.random.uniform(force1_bound[0], force1_bound[1], 1)
    force2 = -np.random.uniform(force2_bound[0], force2_bound[1], 1)
    return torch.tensor([force1, force2], device=dev)

### Precompute the M matrix in the stiff regime>

In [ ]:
alpha_transfer = 100
beta=0.5
p=10

A_transfer = get_A(alpha=alpha_transfer)
force_transfer = force_list[0]
IC_transfer = IC_list[0]

A_transfer = A_transfer.double()
dH_dt_new = dH_dt_new.double()
H = H.double()
H_0 = H_0.double()

M_inv = compute_M_inv(dH_dt_new, H, H_0, t_eval, A_transfer)

if not change_force:
    force_terms = compute_force_term(t_eval, A_transfer, force_transfer, H, dH_dt_new)

if not change_IC:
    IC_term = torch.matmul(H_0.T, IC_transfer)

### Compute 1000 solution with changing IC

In [ ]:
np.random.seed(int(time.time()))
nb_transfer_equation = 1000

IC_transfer_list = [random_IC() for _ in range(nb_transfer_equation)] if change_IC else [IC_transfer for _ in range(nb_transfer_equation)]

force_transfer_list = [random_force() for _ in range(nb_transfer_equation)] if change_force else [force_transfer for _ in range(nb_transfer_equation)]

computational_time = defaultdict(list)
max_error = defaultdict(list)
mean_error = defaultdict(list)
solution = defaultdict(list)

for IC_transfer, force_transfer in tqdm(zip(IC_transfer_list, force_transfer_list), total=nb_transfer_equation):
    # PINNS (change only IC)
    if (change_IC) & (~change_force):
        pinns_sol, _, _, _, total_time = solve_perturbation_TL(beta=beta, p=p, t_eval=t_eval,
                                                               alpha=alpha_transfer, A=A_transfer,
                                                               force=force_transfer, IC=IC_transfer,
                                                               H=H, H_0=H_0, dH_dt=dH_dt_new, dev=dev,
                                                               force_func_index=force_func_index,
                                                               numerical_sol_fct=numerical_sol_fct,
                                                               force_function_PINNS=force_function_PINNS,
                                                               force_function_numerical=force_function_numerical,
                                                               compute_numerical_pert=False,
                                                               numerical_perturbation_fct=numerical_perturbation_fct,
                                                               verbose=False, M_inv=M_inv)
    
    # # PINNS (change only force)
    # elif (change_force) & (~change_IC):
    #     start = time.time()
    #     force_terms = compute_force_term(t_eval, A_transfer, force_transfer, H, dH_dt_new)
    #     rhs_terms = force_terms + IC_term
    #     W_out = torch.matmul(M_inv, rhs_terms)
    #     pinns_sol = torch.matmul(H, W_out)
    #     end = time.time()

    # # PINNS (change IC and force)
    # elif (change_force) & (change_IC):
    #     start = time.time()
    #     force_terms = compute_force_term(t_eval, A_transfer, force_transfer, H, dH_dt_new)
    #     W_out, _ = compute_W_with_IC(M_inv, force_terms, IC_transfer, H_0)
    #     pinns_sol = torch.matmul(H, W_out)
    #     end = time.time()

    solution["PINNS"].append(pinns_sol.T)
    computational_time["PINNS"].append(total_time)

    # solve with numerical methods
    for method, fct in numerical_methods.items():
        start = time.time()
        numerical_sol = fct(t_eval.detach().cpu().numpy().squeeze(),
                            IC_transfer.detach().cpu().squeeze(),
                            alpha_transfer)
        solution[method].append(numerical_sol)
        end = time.time()
        computational_time[method].append(end-start)

### Plot average computational time

In [ ]:
import matplotlib.pyplot as plt

color = {"PINNS": 'orange', "RK45": 'b', "Radau": 'g'}
fig, ax = plt.subplots(1, tight_layout=True, figsize=(8, 4))

height = 0.6  # Change width to height for horizontal bars

for i, (method, compt_time) in enumerate(computational_time.items()):
    ax.barh(i, sum(compt_time)/len(compt_time), height=height, color=color[method], label=f"{method}")
    ax.annotate(f'{sum(compt_time)/len(compt_time):1.2e}', (sum(compt_time)/len(compt_time), i),
                ha='left', va='center', fontsize=12)  # Adjusted annotation placement for horizontal bars

ax.set_xscale("log")  # Change yscale to xscale for horizontal bars
change_title = "IC and force" if (change_force and change_IC) else ("force" if change_force else "IC")
#ax.set_title(rf"Average Time Solving {nb_transfer_equation} stiff equations $\alpha$={alpha_transfer} by changing {change_title}", fontsize=20)
ax.set_xlabel('Time (s)', fontsize=16)  # Change ylabel to xlabel for horizontal bars
ax.set_yticks([i for i in range(len(computational_time))])
ax.set_yticklabels(computational_time.keys())  # Change xticks to yticks and set_xticks to set_yticks
ax.tick_params(axis='y', labelsize=14)
ax.tick_params(axis='x', labelsize=16)
fig.tight_layout()
ax.margins(x=0.15)

plt.show()
